In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

In [2]:
model_path  ='/CV/xhr_project/llm/model/Mistral-7B-Instruct-v0.2'
# model_path2 = '/CV/xhr_project/llm/model/llama2/Llama-2-7b-chat'
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16).to('cuda:0')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def build_input(prompt, history):
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    return messages

In [12]:
def qa(prompt):
    input_str = build_input(prompt=prompt, history=[])
    # print(input_str)
    input_ids = tokenizer.apply_chat_template(input_str, return_tensors="pt").cuda()

    generated_ids = model.generate(input_ids, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id, do_sample=False)
    decoded = tokenizer.batch_decode(generated_ids)
    # print(len(input_ids[0]), len(decoded[0]))

    return decoded[0].split('[/INST]')[-1].replace(tokenizer.eos_token, '').strip()


In [13]:
coin_template_few = "Take a deep breath and work on this problem step-by-step.\
                Here are some expamples: \n \
                Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?\nA: The coin was flipped by Ka and Sherrie. So the coin was flipped 2 times, which is an even number. The coinstarted heads up, so after an even number of flips, it will still be heads up. So the answer is yes.\n\n \
                Q: A coin is heads up. Jamey fips the coin. Teressa flips the coin. Is the coin still heads up?\nA: The coin was flipped by Jamey and Teressa. So the coin was fipped 2 times, which is an even number. Thecoin started heads up, so after an even number of flips, it will still be heads up. So the answer is yes.\n\n \
                O: A coin is heads up. Maybelle fips the coin. Shalonda does not fip the coin. Is the coin still heads up?\nA: The coin was flipped by Maybelle. So the coin was flipped 1 time, which is an odd number. The coin startedheads up, so after an odd number of flips, it will be tails up. So the answer is no.\n\n \
                Q: {question}"

coin_template_non = '{question}'

coin_template_zero = 'Take a deep breath and work on this problem step-by-step.\nQuestion: {question}'

In [14]:
letter_template_few = """Let's think step by step.\
                Here are some expamples: \n \
                Take the last letters of the words in "Elon Musk" and concatenate them.\nA: The last letter of "Elon" is "n". The last letter of "Musk" is "k". Concatenating them is "nk". The answer is nk.\n\n\
                Q: Take the first letters of the words in "Larry Page" and concatenate them.\nA: The first letter of "Larry" is "L". The first letter of "Page" is "'P". Concatenating them is "LP". The answer is LP.\n\n\
                O: Take the second letters of the words in "Sergey Brin" and concatenate them.\nA: The second letter of "Sergey" is "r". The second letter of "Brin" is "'i". Concatenating them is "ri". The answer is ri.\n\n\
                Q: Take the {index} letters of the words in {question} and concatenate them.\nA: """

letter_template_non = "Q: Take the '{index}'' letters of the words in '{question}' and concatenate them.\nA: "

letter_template_zero = "Let's think step by step.\nQ: Take the '{index}' letter of the words in '{question}' and concatenate them.\nA: "

In [15]:
prompt = letter_template_zero.format(index='second', question='Morgan Calderon')
print(qa(prompt))

<s> [INST] Let's think step by step.
Q: Take the 'second' letter of the words in 'Morgan Calderon' and concatenate them.
A:  [/INST] The second letters of each word in "Morgan Calderon" are: o, r, g, a, d, e, r, n. When concatenated, they form the word "orgadern". However, it's important to note that this word doesn't have any known meaning or significance. It's just a result of following the instructions given in the question.</s>


In [49]:
qa(prompt=coin_template_zero.format(question='Q: A coin is heads up. Darnasia flips the coin.  Emryk flips the coin.  Is the coin still heads up?'))

'The information given does not specify whether Darnasia and Emryk flipped the coin heads or tails. In order to determine if the coin is still heads up, we would need to know the outcome of their flips.'

In [46]:
import pandas as pd
from tqdm import tqdm
import random

df = pd.read_csv('dataset/coin_flip.csv')
# filtered_df = df[df['length'] == 7]
# sampled_df7 = filtered_df.sample(n=100, random_state=1)
sampled_df = df.sample(n=200, random_state=1)

In [48]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    # q = row['Content']
    q = row['inputs']
    # index2 = row['ordinal_index']

    # result = qa(prompt=letter_template_zero.format(index=index2, question=q))
    result = qa(prompt=coin_template_zero.format(question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    # print(result)
    results_non.append(result)
    # answer = result.split('\n\n')[0]
    # if answer == 'yes' or 'no':
    #     pass
    # else:
    #     answer = 'unk'
    # if answer == row['targets']:
    #     accuracy += 1


sampled_df.to_csv('output/result_mis_coin_zero.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [10:56,  3.28s/it]


In [168]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['Content']
    index2 = row['ordinal_index']

    result = qa(prompt=letter_template_few.format(index=index2, question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    # print(result)
    results_non.append(result)
    # answer = result.split('\n\n')[0]
    # if answer == 'yes' or 'no':
    #     pass
    # else:
    #     answer = 'unk'
    # if answer == row['targets']:
    #     accuracy += 1


sampled_df.to_csv('output/result_letter_few.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [10:01,  3.01s/it]


In [169]:
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['Content']
    index2 = row['ordinal_index']

    result = qa(prompt=letter_template_non.format(index=index2, question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    # print(result)
    results_non.append(result)
    # answer = result.split('\n\n')[0]
    # if answer == 'yes' or 'no':
    #     pass
    # else:
    #     answer = 'unk'
    # if answer == row['targets']:
    #     accuracy += 1


sampled_df.to_csv('output/result_letter_non.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [03:28,  1.04s/it]


In [110]:
sampled_df = df.sample(n=200, random_state=1)
import csv
results_non = []
for index, row in tqdm(sampled_df.iterrows()):
    q = row['inputs']

    result = qa(prompt=coin_template_non.format(question=q))
    sampled_df.at[index, 'pred'] = result.replace('\n', '\\n').strip()
    results_non.append(result)
    # answer = result.split('\n\n')[0]
    # if answer == 'yes' or 'no':
    #     pass
    # else:
    #     answer = 'unk'
    # if answer == row['targets']:
    #     accuracy += 1


sampled_df.to_csv('output/result_coin_non.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

200it [12:04,  3.62s/it]


In [132]:
def replace_with_ordinal(index):
    ordinal_map = {0: "first", 1: "second", 2: "third", 3: "fourth", 4: "fifth", 5: "sixth", 6: "seventh", 7: "eighth", 8: "ninth", 9: "tenth", -1: "last"}  # 添加更多的序数词，根据需要
    ordinal_index = ordinal_map.get(index, str(index))
    return ordinal_index

In [143]:
import pandas as pd
from tqdm import tqdm
import random

df = pd.read_csv('dataset/full_name.csv')
results = []
for index, row in tqdm(df.iterrows()):
    index2 = row['num']
    
    ordinal_index = replace_with_ordinal(index2)
    # print(f"index: {index} ---convert to ordinal: {ordinal_index}")
    # print()
    df.at[index, 'ordinal_index'] = ordinal_index
    # print(result)
    # results_non.append(result)
columns = list(df.columns)
columns.insert(2, columns.pop(columns.index('ordinal_index')))
df = df.reindex(columns=columns)
df.to_csv('output/full_name_ordinal.csv', index = False, quoting=csv.QUOTE_NONNUMERIC)

0it [00:00, ?it/s]

1000it [00:00, 26878.98it/s]


In [56]:
import json
content = []
with open('/CV/xhr_project/llm/Paper/nips/dataset/grade-school-math-master/grade_school_math/data/test.jsonl', 'r')as file:
    for line in file:
        data = json.loads(line)
        content.append(data)

import random
random.seed(11)
test_set = random.sample(content, 100)

In [57]:
GSM8K_template_non = '{question}'

GSM8K_template_zero = "Let's think step by step. {question}"

In [ ]:
prompt = GSM8K_template_zero.format(question="Colby loves going to the movies and every month his parents give him $150 to spend at the movies. Tickets for Fridays and Saturdays cost $10. Tickets for any other day cost $7. Popcorn costs $8 and boxes of candy cost $2. It is the last day of the month and it's a Friday. He wants to make sure he gets a popcorn and box of candy that night. How many movies can he see if he already saw 5 movies on a Friday or Saturday, 8 movies on other days, had 2 tubs of popcorn, and four boxes of candy that month?")
print(prompt, qa(prompt))

In [59]:
from tqdm import tqdm
for item in tqdm(test_set):
    prompt = GSM8K_template_zero.format(question=item['question'])
    result = qa(prompt)
    item['pred'] = result
    item['ground truth'] = item.pop('answer')


100%|██████████| 100/100 [09:49<00:00,  5.90s/it]


In [60]:
test_set[0]

{'question': "Colby loves going to the movies and every month his parents give him $150 to spend at the movies. Tickets for Fridays and Saturdays cost $10. Tickets for any other day cost $7. Popcorn costs $8 and boxes of candy cost $2. It is the last day of the month and it's a Friday. He wants to make sure he gets a popcorn and box of candy that night. How many movies can he see if he already saw 5 movies on a Friday or Saturday, 8 movies on other days, had 2 tubs of popcorn, and four boxes of candy that month?",
 'pred': "Let's first find out how much Colby spent on movies, popcorn, and candy up to this point in the month.\n\n1. Movies:\n   - 5 Fridays or Saturdays: 5 * $10 = $50\n   - 8 other days: 8 * $7 = $56\n   - Already saw: 5 * $10 (Fridays or Saturdays) = $50\n   - Total movies: 5 (Fridays or Saturdays) + 8 (other days) = 13\n   - Spent on movies: $50 (Fridays or Saturdays) + $56 (other days) = $106\n\n2. Popcorn:\n   - 2 tubs: $8 * 2 = $16\n\n3. Candy:\n   - 4 boxes: $2 * 4 

In [61]:

import json

# 指定要写入的文件名
output_file = "output/GSM8K_mis_zero.jsonl"

# 打开文件以写入模式
with open(output_file, 'w') as f:
    # 遍历 JSON 数据列表
    for json_data in test_set:
        # 将每个 JSON 对象转换为字符串，并添加换行符
        json_line = json.dumps(json_data) + '\n'
        # 将 JSON 对象写入文件
        f.write(json_line)

print("JSONL 文件已创建：", output_file)

JSONL 文件已创建： output/GSM8K_mis_zero.jsonl
